In [1]:
# NEED TO MAKE SURE WE ARE IN CORRECT DIRECTORY FOR NOTEBOOK
import os
if os.getcwd() == 'c:\\Users\\jafar\\Desktop\\V2022\\Master2022\\UserSimCRS\\usersimcrs':
  os.chdir("..") # Change to root level directory
os.getcwd()


'c:\\Users\\jafar\\Desktop\\V2022\\Master2022\\UserSimCRS'

In [2]:
# IMPORTS
"""Console application for running simulation."""

import argparse
import configparser
import os
import sys
import json
import yaml
import datetime

from dialoguekit.nlu.models.diet_classifier_rasa import IntentClassifierRasa
from dialoguekit.nlu.models.intent_classifier_cosine import IntentClassifierCosine
from dialoguekit.nlg.nlg import NLG
from dialoguekit.nlu.nlu import NLU
from dialoguekit.agent.agent import Agent
from dialoguekit.core.dialogue import Dialogue
from dialoguekit.core.ontology import Ontology
from dialoguekit.core.recsys.item_collection import ItemCollection
from dialoguekit.core.recsys.ratings import Ratings
from dialoguekit.manager.dialogue_manager import DialogueManager
from dialoguekit.platforms.platform import Platform
from dialoguekit.agent.terminal_agent import TerminalAgent
from dialoguekit.agent.moviebot_agent import MovieBotAgent
from dialoguekit.core.intent import Intent
from dialoguekit.core.utterance import Utterance
from dialoguekit.nlu.models.satisfaction_classifier import SatisfactionClassifier

from usersimcrs.simulator.user_simulator import UserSimulator
from usersimcrs.sample_agent.sample_agent import SampleAgent
from usersimcrs.simulator.agenda_based_simulator import AgendaBasedSimulator
from usersimcrs.simulator.preference_model import (
    PreferenceModel,
    PreferenceModelVariant,
)
from usersimcrs.simulator.interaction_model import InteractionModel
from usersimcrs.utils.persona_generator import Persona, Context


In [3]:
# FUNCTIONS
def simulate_conversation(
    agent: Agent, user_simulator: UserSimulator
) -> Dialogue:
    """Simulates a single conversation and returns the resulting dialogue.

    Args:
        agent: An agent.
        user_simulator: A user simulator.
    """
    platform = Platform()  # TODO: Add simulator platform
    dm = DialogueManager(agent, user_simulator, platform)
    agent.connect_dialogue_manager(dialogue_manager=dm)
    user_simulator.connect_dialogue_manager(dialogue_manager=dm)
    dm.start()
    dm.close()
    return dm.dialogue_history

In [4]:
# READ CONFIG
conf_parser = argparse.ArgumentParser(
    description=__doc__, # printed with -h/--help
    # Don't mess with format of description
    formatter_class=argparse.RawDescriptionHelpFormatter,
    # Turn off help, so we print all options in response to -h
    add_help=False
    )
conf_file = "configs\config.ini"
cp = configparser.ConfigParser()
cp.read([conf_file])
input_file_paths = dict(cp.items('SETTINGS'))

In [5]:
# INITIALISE OBJECTS
# agent = SampleAgent(agent_id="sample_agent")
# agent = TerminalAgent("test")

ontology = Ontology(input_file_paths["ontology"])

item_collection = ItemCollection()
item_collection.load_items_csv(input_file_paths["items"],["ID", "NAME", "genres","keywords"])

ratings = Ratings(item_collection=item_collection)
ratings.load_ratings_csv(input_file_paths["ratings"])

with open(input_file_paths["dialogues"]) as annotated_dialogues_file:
  annotated_conversations = json.load(annotated_dialogues_file)
gt_intents = []
utterances = []
for conversation in annotated_conversations:
    for turn in conversation["conversation"]:
        if turn["participant"] == "AGENT":
            gt_intents.append(Intent(turn["intent"]))
            utterances.append(Utterance(turn["utterance"]))
with open(input_file_paths["ir"]) as yaml_file:
  config = yaml.load(yaml_file, Loader = yaml.FullLoader)
agent_intents = [Intent(x) for x in config["agent_intents"]]

interaction_model = InteractionModel(
  input_file_paths["ir"], annotated_conversations
)

satisfaction_model = SatisfactionClassifier()

nluc = IntentClassifierCosine(agent_intents)
nluc.train_model(utterances,gt_intents)

c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.24.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
# TRAIN OBJECTS THAT REQUIRE IT
nlur = IntentClassifierRasa(
  agent_intents,
  "data/agents/moviebot/annotated_dialogues_v2_minimal_rasa_agent.yaml",
  ".rasa",
)
nlur.train_model()

nlu = NLU(nluc,[nlur])
nlg = NLG()
nlg.template_from_file(template_file=input_file_paths["dialogues"], participant_to_learn="USER", satisfaction_classifier= satisfaction_model)

c:\Users\jafar\anaconda3\envs\usersimcrs\lib\site-packages\rasa\utils\train_utils.py:530: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss.
  category=UserWarning,


WhitespaceTokenizer_0
CountVectorsFeaturizer_1


Epochs: 100%|██████████| 300/300 [01:53<00:00,  2.64it/s, t_loss=1.39, i_acc=1, e_f1=0.956]


In [116]:
# COPY agenda for fair simulations
from copy import deepcopy
intmodel = deepcopy(interaction_model)

In [ ]:
# STATIC AGENDA - 20 CONV PER PERSONA-CONTEXT PAIR.
personas = []
coops = [0.2,0.6,1.0]
totd = [(datetime.time(0,0,0),datetime.time(3,0,0)),(datetime.time(21,0,0),datetime.time(23,59,59)),(datetime.time(15,0,0),datetime.time(18,0,0))]
init_satisfaction = 3.0
names = ["Sarah Jackson","Tyreson Tyre","Jefferson Frank"]
agent = MovieBotAgent("MovieBot","http://192.168.0.168:5001/")
errors_during_simulations = []
for i,name in enumerate(names):
  ctx = Context(group_setting=True, time_of_the_day=totd[2],weekend=True)
  persona = Persona(name=name, id=str(i*2 + 13), satisfaction=init_satisfaction,cooperativeness=coops[i],context=ctx)
  persona.calculate_max_retries()
  preference_model = PreferenceModel(
    ontology,
    item_collection,
    ratings,
    PreferenceModelVariant.PKG,
    historical_user_id=persona.id
  )
  for j in range(20):
    while True:
        intmodel = deepcopy(interaction_model)
        try:
            simulator = AgendaBasedSimulator(
            "simulator",
            preference_model,
            intmodel,
            nlu,
            nlg,
            ontology,
            item_collection,
            ratings,
            persona,
            satisfaction_model
        )
            simulate_conversation(agent,simulator)
        except Exception as e:
            errors_during_simulations.append(e)
            continue
        break
  

In [ ]:
# CHECKING IF INTENTS IN THE SIMULATED DIALOGUES AGREE WITH THE INTENT SCHEME IN USE.
from dialoguekit.utils.dialogue_reader import json_to_dialogues
from dialoguekit.core.dialogue import DialogueParticipant
intent_set = set()
dialogues = json_to_dialogues("dialogue_export\same_agenda_different_persona_context\gsTweT\\t21_24\MovieBot_13.json", agent_id="MovieBot",user_id="13")
for dialogue in dialogues:
  for utterance in dialogue.utterances:
    if utterance["sender"] == DialogueParticipant.AGENT:
      intent_set.add(utterance["utterance"].intent.label)
print(list(intent_set))

intent_set = set()
dialogues = json_to_dialogues("dialogue_export\same_agenda_different_persona_context\gsTweT\\t21_24\MovieBot_15.json", agent_id="MovieBot",user_id="15")
for dialogue in dialogues:
  for utterance in dialogue.utterances:
    if utterance["sender"] == DialogueParticipant.AGENT:
      intent_set.add(utterance["utterance"].intent.label)
print(list(intent_set))

intent_set = set()
dialogues = json_to_dialogues("dialogue_export\same_agenda_different_persona_context\gsTweT\\t21_24\MovieBot_17.json", agent_id="MovieBot",user_id="17")
for dialogue in dialogues:
  for utterance in dialogue.utterances:
    if utterance["sender"] == DialogueParticipant.AGENT:
      intent_set.add(utterance["utterance"].intent.label)
print(list(intent_set))